In [1]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd
pd.set_option('display.max_rows', 10)
!date

%load_ext autoreload
%autoreload 2

Wed Jun 12 10:58:19 PDT 2019


# Load IPUMS 1940 Census Microdata, and filter to just WA state

Codebook: https://usa.ipums.org/usa/resources/1940CensusDASTestData/EXT1940USCB.cbk

My previous approach failed, because the person rows do not include the location details.  Annoying!  How to address?  First make a household file, then make a person file, getting the location details from the household file.

In [2]:
with open('data/full_ipums_1940_census_colspecs.txt') as f:
    colspec_str = f.read()

In [3]:
colspecs = pd.Series()
for line in colspec_str.split('\n'):
    name = line.split()[0].strip()
    val_str = line.split()[2]
    
    start_str = val_str.split('-')[0]
    start_index = int(start_str) - 1

    end_str = val_str.split('-')[-1]
    end_index = int(end_str)
    
#     print(name, val_str, (start_index, end_index))
    colspecs[name] = (start_index, end_index)

In [4]:
hh = pd.read_csv('/home/j/Project/Models/us_census/hh_1940_raw.csv', index_col=0)
hh = hh.loc[:, ['SERIAL', 'STATEFIP', 'COUNTY', 'ENUMDIST', 'GQ']]
hh

/ihme/homes/abie/.local/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,SERIAL,STATEFIP,COUNTY,ENUMDIST,GQ
0,1,2,3000,10,1
3,2,2,3000,10,1
8,3,2,3000,10,1
10,4,2,3000,10,1
12,5,2,3000,10,1
...,...,...,...,...,...
170897716,38492959,56,455,10,1
170897718,38492960,56,455,10,1
170897722,38492961,56,455,10,6
170897724,38492962,56,455,10,6


In [5]:
hh = hh[hh.STATEFIP == 53]  # just WA state
hh.shape

(611156, 5)

# Pass 2: Generate Person file for WA state

Since I have not been able to do this successfully on a single machine, I will try using dask to parallelize.

In [6]:
# %%time

# fname = '/home/j/Project/Models/us_census/EXT1940USCB.dat'

# iter = pd.read_fwf(fname, list(colspecs), header=None, names=colspecs.index,
#                    chunksize=1_000_000)
# df_list = []

# for i, df_i in enumerate(iter):
#     if i % 32 == 0:
#         print(f'Processing chunk {i}, a dataframe with shape {df_i.shape}', flush=True)
#     else:
#         print('.', flush=True, end='')
#     df_i = df_i[df_i.RECTYPE == 'P'].loc[:, ['SERIAL', 'AGE', 'HISPAN', 'RACE']]
#     df_i = pd.merge(df_i, hh, on='SERIAL', how='inner')

#     df_list.append(df_i)

In [7]:
from dask_jobqueue import SGECluster

cluster = SGECluster(queue='all.q',
                     project='ihme_general',
                     resource_spec='m_mem_free=20G,fthread=2,archive=True',  # for memory requests, this also needs to be specified
                     processes=2,   # we request X processes per worker
                     memory='20GB',  # for memory requests, this must be specified
                     cores=2, # cores different than processes?
                     )

In [8]:
cluster.scale(128)
cluster

In [24]:
from dask.distributed import Client
client = Client(cluster)  # Connect this local process to remote workers
client

Client Scheduler: tcp://10.158.96.178:46249,Cluster Workers: 6 Cores: 6 Memory: 60.00 GB


In [25]:
fname = '/home/j/Project/Models/us_census/EXT1940USCB.dat'

import dask.dataframe as dd

df = dd.read_fwf(fname, header=None, colspecs=list(colspecs), names=colspecs.index)
df

,RECTYPE,YEAR,DATANUM,SERIAL,NUMPREC,SUBSAMP,HHWT,NUMPERHH,HHTYPE,DWELLING,SLPERNUM,CPI99,REGION,STATEICP,STATEFIP,COUNTY,URBAN,METRO,METAREA,METAREAD,CITY,CITYPOP,SIZEPL,URBPOP,SEA,WARD,CNTRY,GQ,GQTYPE,GQTYPED,GQFUNDS,FARM,OWNERSHP,OWNERSHPD,RENT,VALUEH,NFAMS,NSUBFAM,NCOUPLES,NMOTHERS,NFATHERS,MULTGEN,MULTGEND,ENUMDIST,SUPDIST,RESPOND,SPLIT,SPLITHID,SPLITNUM,SPLIT40,SERIAL40,NUMPREC40,EDMISS,RECTYPEP,YEARP,DATANUMP,SERIALP,PERNUM,PERWT,SLWT,SLREC,RESPONDT,FAMUNIT,FAMSIZE,SUBFAM,SFTYPE,SFRELATE,MOMLOC,STEPMOM,MOMRULE_HIST,POPLOC,STEPPOP,POPRULE_HIST,SPLOC,SPRULE_HIST,NCHILD,NCHLT5,NSIBS,ELDCH,YNGCH,RELATE,RELATED,SEX,AGE,AGEMONTH,MARST,MARRNO,AGEMARR,CHBORN,RACE,RACED,HISPAN,HISPAND,BPL,BPLD,MBPL,MBPLD,FBPL,FBPLD,NATIVITY,CITIZEN,MTONGUE,MTONGUED,SPANNAME,HISPRULE,SCHOOL,HIGRADE,HIGRADED,EDUC,EDUCD,EMPSTAT,EMPSTATD,LABFORCE,OCC,OCC1950,IND,IND1950,CLASSWKR,CLASSWKRD,WKSWORK1,WKSWORK2,HRSWORK1,HRSWORK2,DURUNEMP,UOCC,UOCC95,UIND,UCLASSWK,INCWAGE,INCNONWG,OCCSCORE,SEI,PRESGL,ERSCOR50,EDSCOR50,NPBOSS50,MIGRATE5,MIGRATE5D,MIGPLAC5,MIGMET5,MIGTYPE5,MIGCITY5,MIGSEA5,SAMEPLAC,SAMESEA5,MIGCOUNTY,VETSTAT,VETSTATD,VET1940,VETWWI,VETPER,VETCHILD,HISTID,SURSIM,SSENROLL
npartitions=664,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,object,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,object,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [26]:
hh = dd.read_csv('/home/j/Project/Models/us_census/hh_1940_raw.csv')
hh = hh.loc[:, ['SERIAL', 'STATEFIP', 'COUNTY', 'ENUMDIST', 'GQ']]

hh = hh[hh.STATEFIP == 53]  # just WA state
hh.shape

(Delayed('int-999538fd-aff3-4781-952f-12f67e99ebc4'), 5)

In [28]:
hh.shape[0].compute()

611156

In [29]:
df_person = df[df.RECTYPE == 'P'].loc[:, ['SERIAL', 'AGE', 'HISPAN', 'RACE']]
df_wa_person = dd.merge(df_person, hh, on='SERIAL', how='inner')

In [30]:
%%time

len(df_wa_person)

CPU times: user 1min 47s, sys: 6.64 s, total: 1min 54s
Wall time: 6min 21s


1741616

In [31]:
%%time

df_wa_person.head()

CPU times: user 52.4 s, sys: 4.89 s, total: 57.3 s
Wall time: 6min


,SERIAL,AGE,HISPAN,RACE,STATEFIP,COUNTY,ENUMDIST,GQ
0,36437747,34,0,1,53,30,40,1
1,36437747,15,0,1,53,30,40,1
2,36438380,42,0,1,53,30,70,1
3,36438380,37,0,1,53,30,70,1
4,36438380,16,0,1,53,30,70,1


In [32]:
%%time

computed_df = df_wa_person.compute()

CPU times: user 1min 52s, sys: 7.61 s, total: 2min
Wall time: 6min 17s


In [35]:
computed_df

,SERIAL,AGE,HISPAN,RACE,STATEFIP,COUNTY,ENUMDIST,GQ
0,36437747,34,0,1,53,30,40,1
1,36437747,15,0,1,53,30,40,1
2,36438380,42,0,1,53,30,70,1
3,36438380,37,0,1,53,30,70,1
4,36438380,16,0,1,53,30,70,1
...,...,...,...,...,...,...,...,...
2527,37045558,36,0,1,53,530,910,1
2528,37045692,43,0,1,53,530,920,1
2529,37045692,34,0,1,53,530,920,1
2530,37046102,24,0,1,53,530,930,1


In [33]:
# save relevant rows for comparison
computed_df.to_csv('/home/j/Project/Models/us_census/per_1940_raw.csv')

In [38]:
!ls -halt /home/j/Project/Models/us_census/per_1940_raw.csv
!head /home/j/Project/Models/us_census/per_1940_raw.csv

-rw-r--r-- 1 abie Domain Users 56M Jun 12 11:51 /home/j/Project/Models/us_census/per_1940_raw.csv
,SERIAL,AGE,HISPAN,RACE,STATEFIP,COUNTY,ENUMDIST,GQ
0,36437747,34,0,1,53,30,40,1
1,36437747,15,0,1,53,30,40,1
2,36438380,42,0,1,53,30,70,1
3,36438380,37,0,1,53,30,70,1
4,36438380,16,0,1,53,30,70,1
5,36438701,47,0,1,53,30,80,1
6,36438701,41,0,1,53,30,80,1
7,36438701,13,0,1,53,30,80,1
8,36438701,9,0,1,53,30,80,1


In [42]:
computed_df.shape

(1741616, 8)

In [43]:
computed_df.STATEFIP.value_counts()

53    1741616
Name: STATEFIP, dtype: int64

In [44]:
computed_df.GQ.value_counts()

1    1683145
6      20640
4      17296
3      12044
2       8491
Name: GQ, dtype: int64

In [45]:
cluster.close()